In [ ]:
# 33
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=chatgpt&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_ChatGPT_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

正在爬取頁面: 35it [22:41, 38.89s/it]


📄 第 36 頁無課程內容，爬蟲結束。

✅ 爬取完成，資料已儲存至：coursera_ChatGPT_TS.csv


In [ ]:
# 62
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=coding&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_coding_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 186
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=generative%20ai&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_GenerativeAi_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 50
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=cybersecurity&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_CyberSecurity_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 33
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=devops&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_devops_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 72
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=ethical%20hacking&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_EthicalHacking_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 55
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=java&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_java_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 649
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=web%20development&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_WebDevelopment_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 280
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=computer%20science&page={page}&topic=Computer%20Science"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_ComputerScience_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")

In [ ]:
# 102
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
}

all_data = []


def get_extra_info(course_url):
    start_time = ""
    suggested_study_time = ""
    study_duration = ""
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        start_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-za0201.cds-10.cds-11.cds-grid-item > div.cds-9.css-0.cds-11.cds-grid-item.cds-56 > div > div > div > div.cds-9.css-0.cds-11.cds-grid-item.cds-56.cds-79 > div.css-y6ppwi > div > div > div > form > button > span > div > div > span"
        )
        suggested_study_time_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fk6qfz"
        )
        study_duration_element = soup.select_one(
            "#rendered-content > div > main > section.css-oe48t8 > div > div > div.cds-9.css-xtzux6.cds-11.cds-grid-item > div > div.css-2qp4i2.cds-168 > div:nth-child(4) > div > div > div.css-fw9ih3"
        )
        start_time = start_time_element.text.strip() if start_time_element else ""
        suggested_study_time = (
            suggested_study_time_element.text.strip()
            if suggested_study_time_element
            else ""
        )
        study_duration = (
            study_duration_element.text.strip() if study_duration_element else ""
        )
    except Exception as e:
        print(f"⚠️ 擷取課程額外資訊失敗：{course_url}，錯誤：{e}")
    return start_time, suggested_study_time, study_duration


def get_course_details(course_url):
    details = {
        "課程": "",
        "技能": "",
        "課程資訊": "",
        "師資": "",
        "開課時間": "",
        "建議學習時間": "",
        "學習時長": "",
    }
    try:
        resp = requests.get(course_url, headers=headers, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        course_title = soup.find(
            "a", class_="cds-119 cds-113 cds-115 css-17cxvu3 cds-142", href="#modules"
        )
        details["課程"] = course_title.text.strip() if course_title else ""
        skills = []
        skill_spans = soup.find_all("span", {"class": "css-1l1jvyr"})
        for span in skill_spans:
            skill_link = span.find(
                "a", {"class": "cds-119 cds-113 cds-115 css-113xph7 cds-142"}
            )
            if skill_link:
                skills.append(skill_link.get_text(strip=True))
        details["技能"] = ", ".join(skills) if skills else ""
        modules = []
        for mod in soup.find_all("h3", class_="cds-119 css-1pxm1ir cds-121"):
            modules.append(mod.text.strip())
        details["課程資訊"] = " | ".join(modules)
        instructors = []
        for span in soup.find_all("span", class_="css-6ecy9b"):
            name = span.text.strip()
            if (
                name not in instructors
                and len(name.split()) >= 2
                and not any(
                    keyword in name.lower()
                    for keyword in [
                        "star",
                        "more",
                        "access",
                        "certificate",
                        "refund",
                        "?",
                    ]
                )
            ):
                instructors.append(name)
        details["師資"] = ", ".join(instructors)
        start_time, suggested_study_time, study_duration = get_extra_info(course_url)
        details["開課時間"] = start_time
        details["建議學習時間"] = suggested_study_time
        details["學習時長"] = study_duration
    except Exception as e:
        print(f"⚠️ 詳情頁連線失敗：{course_url}，錯誤：{e}")
    return details


page = 1
pbar = tqdm(desc="正在爬取頁面", ncols=100)
while True:
    url = f"https://www.coursera.org/courses?query=python&page={page}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"\n⚠️ 第 {page} 頁連線失敗：{e}")
        break
    soup = BeautifulSoup(response.content, "html.parser")
    course_cards = soup.find_all("div", class_="cds-ProductCard-content")
    if not course_cards:
        print(f"\n📄 第 {page} 頁無課程內容，爬蟲結束。")
        break
    for card in course_cards:
        title_elem = card.find("h3", class_="cds-CommonCard-title css-6ecy9b")
        title = title_elem.text.strip() if title_elem else "N/A"
        rating_elem = card.find("div", class_="css-1whdyhf")
        rating = (
            rating_elem.text.strip().replace("Rating, ", "") if rating_elem else "N/A"
        )
        review = "N/A"
        review_elems = card.find_all("div", class_="css-vac8rf")
        for elem in review_elems:
            if "reviews" in elem.text.lower():
                review = elem.text.strip()
                break
        meta_elem = card.find("div", class_="cds-CommonCard-metadata")
        meta = (
            meta_elem.find("p", class_="css-vac8rf").text.strip()
            if meta_elem and meta_elem.find("p", class_="css-vac8rf")
            else "N/A"
        )
        link_elem = card.find("a", class_="cds-CommonCard-titleLink")
        if link_elem and link_elem.get("href"):
            course_url = "https://www.coursera.org" + link_elem.get("href")
        else:
            course_url = ""
        if course_url:
            detail_data = get_course_details(course_url)
        else:
            detail_data = {
                "課程": "",
                "技能": "",
                "課程資訊": "",
                "師資": "",
                "開課時間": "",
                "建議學習時間": "",
                "學習時長": "",
            }
        row = {
            "課程名稱": title,
            "評分": rating,
            "評論數": review,
            "Metadata": meta,
            "課程網址": course_url,
            "課程": detail_data["課程"],
            "技能": detail_data["技能"],
            "課程資訊": detail_data["課程資訊"],
            "師資": detail_data["師資"],
            "開課時間": detail_data["開課時間"],
            "建議學習時間": detail_data["建議學習時間"],
            "學習時長": detail_data["學習時長"],
        }
        all_data.append(row)
    page += 1
    pbar.update(1)
    time.sleep(3)
pbar.close()

df = pd.DataFrame(all_data)
output_file = "coursera_python_TS.csv"
try:
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 爬取完成，資料已儲存至：{output_file}")
except PermissionError:
    print(f"\n❌ 無法寫入 '{output_file}'，請關閉檔案或選擇其他路徑。")
except Exception as e:
    print(f"\n❌ 發生錯誤：{str(e)}")